# All Policies

Analyze the performance of our Whittle and Adaptive Policies

In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import json 
import argparse 
import sys
import secrets
from itertools import combinations
import csv
import pandas as pd

In [3]:
from rmab.whittle_policies import *
from rmab.baseline_policies import *
from rmab.mcts_policies import *
from rmab.utils import get_save_path, delete_duplicate_results, restrict_resources
from rmab.simulator import RMABSimulator, run_heterogenous_policy, get_discounted_reward, create_random_transitions
from rmab.fr_dynamics import get_all_transitions, get_db_data, get_all_transitions_partition
from rmab.compute_whittle import get_q_vals,Q_multi_prob, fast_Q_multi_prob
from rmab.multiple_notifications import get_notification_text, get_volunteer_trip_info, get_donor_info, get_recipient_info

/usr0/home/naveenr/miniconda3/envs/food/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
is_jupyter = 'ipykernel' in sys.modules

In [10]:
csv.load

<module 'csv' from '/usr0/home/naveenr/miniconda3/envs/food/lib/python3.8/csv.py'>

In [15]:
csv_name = "labeled_data.csv"
parameters = {
    'model': 'human',
}
df = pd.read_csv("../../results/annotations/"+csv_name)

In [25]:
data = {}
data['parameters'] = {
    'model': 'none', 
    'prompts': 'original'
}
baseline_df = deepcopy(df)
baseline_df = baseline_df.drop(['inadequate_food_jingwu',	'donor_contact_problem_jingwu','recipient_problem_jingwu','info_update_jingwu','inadequate_food_naveen','donor_contact_problem_naveen','recipient_problem_naveen','info_update_naveen'],axis=1)
list_of_baseline_dicts = baseline_df.to_dict(orient='records')
data['records'] = list_of_baseline_dicts
data['annotations'] = []
json.dump(data,open("../../results/annotations/base_examples.json","w"))

In [27]:
data = {}
data['parameters'] = {
    'model': 'naveen', 
    'prompts': 'original'
}
list_of_dicts = df.to_dict(orient='records')
list_of_dicts = [{
    'id': i['id'],
    'inadequate_food': i['inadequate_food_naveen'],
    'donor_problem': i['donor_contact_problem_naveen'],
    'recipient_problem': i['recipient_problem_naveen'], 
    'info_update': i['info_update_naveen']
} for i in list_of_dicts]
data['records'] = []

data['annotations'] = list_of_dicts
json.dump(data,open("../../results/annotations/naveen_annotations.json","w"))

In [28]:
data = {}
data['parameters'] = {
    'model': 'jingwu', 
    'prompts': 'original'
}
list_of_dicts = df.to_dict(orient='records')
list_of_dicts = [{
    'id': i['id'],
    'inadequate_food': i['inadequate_food_jingwu'],
    'donor_problem': i['donor_contact_problem_jingwu'],
    'recipient_problem': i['recipient_problem_jingwu'], 
    'info_update': i['info_update_jingwu']
} for i in list_of_dicts]
data['records'] = []

data['annotations'] = list_of_dicts
json.dump(data,open("../../results/annotations/jingwu_annotations.json","w"))

In [31]:
csv_name = "labeled_data_with_llm.csv"
df = pd.read_csv("../../results/annotations/"+csv_name)

data = {}
data['parameters'] = {
    'model': 'gpt-3.5 turbo', 
    'prompts': 'original'
}
list_of_dicts = df.to_dict(orient='records')
list_of_dicts = [{
    'id': i['id'],
    'inadequate_food': int(i['inadequate_food']),
    'donor_problem': int(i['donor_contact_problem']),
    'recipient_problem': int(i['recipient_problem']), 
    'info_update': int(i['info_update']),
} for i in list_of_dicts]
data['records'] = []

data['annotations'] = list_of_dicts
json.dump(data,open("../../results/annotations/gpt_35_annotations.json","w"))

In [33]:
csv_name = "labeled_data_with_llm_2.csv"
df = pd.read_csv("../../results/annotations/"+csv_name)

data = {}
data['parameters'] = {
    'model': 'gpt-3.5 turbo', 
    'prompts': 'updated'
}
list_of_dicts = df.to_dict(orient='records')
list_of_dicts = [{
    'id': i['id'],
    'inadequate_food': int(i['inadequate_food']),
    'donor_problem': int(i['donor_problem']),
    'recipient_problem': int(i['recipient_problem']), 
    'info_update': int(i['info_update']),
} for i in list_of_dicts]
data['records'] = []

data['annotations'] = list_of_dicts
json.dump(data,open("../../results/annotations/gpt_35_annotations_updated.json","w"))

In [57]:
csv_name = "labeled_data_with_llm_gpt_4.csv"
df = pd.read_csv("../../results/annotations/"+csv_name)

data = {}
data['parameters'] = {
    'model': 'gpt-4 turbo', 
    'prompts': 'updated'
}
list_of_dicts = df.to_dict(orient='records')
list_of_dicts = [{
    'id': i['id'],
    'inadequate_food': int(i['inadequate_food']),
    'donor_problem': int(i['donor_problem']),
    'recipient_problem': int(i['recipient_problem']), 
    'info_update': int(np.nan_to_num(i['info_update'],nan=-1)),
} for i in list_of_dicts]
data['records'] = []

data['annotations'] = list_of_dicts
json.dump(data,open("../../results/annotations/gpt_4_annotations.json","w"))

In [58]:
data = """id	label_comment	inadequate_food_consensus	donor_contact_problem_consensus	recipient_problem_consensus	info_update_consensus
411243	Should be both donor + recipient	1	1	1	1
394294		1	0	1	0
400184		0	0	1	1
410305		0	0	1	1
360938		0	0	1	0
404598		0	0	1	0
400179	Inadequate food is on the part of the donor	0	0	1	0
404770		0	0	1	0
402687		1	1	0	0
387448		1	1	0	0
392579	Inadquate food even if someone else had taken the food before. Make sure info update is only to improve instructions. 	1	1	0	0
410368		0	0	0	1
405044		0	1	0	0
403887	Maybe add a new task for donor problems that aren't just contact	0	1	0	0
393188	Maybe make it explicit when inadequate food	0	1	0	0
373544		1	1	0	0
406379		0	1	0	0
393865		0	1	0	0
408384		0	1	0	0
409731		0	1	0	0
412269	Donor problem not contact problem	1	0	0	0
410448	Donor problem not contact problem	0	1	0	0
404356	Can't really understand comment	1	0	0	0
395770	Not an info update, but something needs to be changed on app	1	0	0	1
375091	Donor problem not contact problem	1	1	0	0
347413		1	0	0	0
373313		1	0	0	0
373689		0	0	0	0
373308	Donor problem at a minimum	1	1	0	0
413265	Misreading	1	0	0	0
411702		1	0	0	0
350847		1	0	0	0
358597		1	0	0	0
349174		1	0	0	0
349991		1	0	0	0
352507		1	0	0	0
357845		1	0	0	0
405069		1	0	0	0
392744		1	0	0	0
359930		1	0	0	0
403500		1	0	0	0
403573		1	0	0	0
349889		1\t0\t0\t0
405060		1	0	0	0
409273		1	0	0	0
392220		1	0	0	0
375090		1	0	0	0
355975		1	0	0	0
373539		1	0	0	0
343870		1	0	0	0
351464		1	0	0	0
394375		1	0	0	0
393128		1	0	0	0
403629		1	0	0	0
395048		1	0	0	0
358151		1	0	0	0
350846		1	0	0	0
395526		1	0	0	0
405073		1	0	0	0
393814		1	0	0	0
392694		1	0	0	0
393228		1	0	0	0
352127		1	0	0	0
375243		1	0	0	0
392745		1	0	0	0
412257		1	0	0	0
394105		1	0	0	0
403534		1	0	0	0
411100		1	0	0	0
356753		1	0	0	0
395559		1	0	0	0
351037		1	0	0	0
409528		1	0	0	0
361155		1	0	0	0
393854		1	0	0	0
357410		1	0	0	0
351370		1	0	0	0
393318		1	0	0	0
403525		1	0	0	0
392239		1	0	0	0
392725		1	0	0	0
394039		1	0	0	0
403531		1	0	0	0
387711		1	0	0	0
411072		1	0	0	0
403571		1	0	0	0
131667		1	0	0	0
343585	Recipient problem if they can't take it all	0	0	1	0
349606		1	0	0	0
392449		1	0	0	0
411109		1	0	0	0
403492		1	0	0	0
393959		1	0	0	0
393960		1	0	0	0
387102		0	0	0	1
373459		0	0	0	1
403746	App update, not an instruction update	0	0	0	1
413351		0	0	0	1
347036	Nothing food rescue people can do about this	0	0	0	0
403595		0	0	0	1
414262	Info update should include how heavy things are, etc.	0	0	0	1
393616		0	0	0	0
393979		0	0	0	1
411292		0	0	0	1
395458	App update, not an instruction update	0	0	0	1
411380	Info update should include contact updates	0	0	0	1
392984		0	0	0	0
405034		0	0	0	0
375218		0	0	0	0
403181		0	0	0	0
374548		0	0	0	0
373324		0	0	0	0
373658		0	0	0	0
374523		0	0	0	0
373610		0	0	0	0
373657		0	0	0	0
373323		0	0	0	0
393138		0	0	0	0
375069		0	0	0	0
391999		0	0	0	0
374524		0	0	0	0
393834		0	0	0	0
403959		0	0	0	0
403668		0	0	0	0
404589		0	0	0	0
394059		0	0	0	0
414540		0	0	0	1
411263		0	0	0	0
414114		0	0	0	0
374988		0	0	0	0
375608		0	0	0	0
373631		0	0	0	0
414403		0	0	0	0
343679		0	0	0	0
405042		0	0	0	0
405141		0	0	0	0
410369		0	0	0	0
409732		0	0	0	0
387418		0	0	0	0
391792		0	0	0	0
412308		0	0	0	0
387301		0	0	0	0
387466		0	0	0	0
387522		0	0	0	0
403643		0	0	0	0
413418		0	0	0	0
414397		0	0	0	0
358598		0	0	0	0
395440		0	0	0	0
360851		0	0	0	0
411232		0	0	0	0
414395		0	0	0	0
403795		0	0	0	0
357715		0	0	0	0
414175		0	0	0	0
345430		0	0	0	0
411238		0	0	0	0
411240		0	0	0	0
356322		0	0	0	0
414409		0	0	0	0
345431		0	0	0	0
356538		0	0	0	0
359299		0	0	0	0
361680		0	0	0	0
411247		0	0	0	0
395466		0	0	0	0
375322		0	0	0	0
413281		0	0	0	0
414281		0	0	0	0
394010		0	0	0	0
414315		0	0	0	0
357453		0	0	0	0
393884		0	0	0	0
392059		0	0	0	0
414191		0	0	0	0
373470		0	0	0	0
407976		0	0	0	0
402149		0	0	0	0
408314		0	0	0	0
396235		0	0	0	0
373611		0	0	0	0
393110		0	0	0	0
395427		0	0	0	0
400169		0	0	0	0
375701		0	0	0	0
373814		0	0	0	0
411223		0	0	0	0
410912	Unclear comment	0	0	1	0
414322		0	0	0	0
411168		0	0	0	0
393358		0	0	0	0
375053		0	0	0	0
406821		0	0	0	0
375609		0	0	0	0
403640		0	0	0	0
404110		0	0	0	0
403619		0	0	0	0
414231	Misreading	0	0	0	0
393914		0	0	0	0
414183		0	0	0	0
375146		0	0	0	0
374179		0	0	0	0
403165		0	0	0	0
402721		0	0	0	0
403284		0	0	0	0
402784		0	0	0	0
356235		0	0	0	0
404440		0	0	0	0
404464		0	0	0	0
356236		0	0	0	0
403769		0	0	0	0
348641	Recipient didn't show up, so it was problem	0	0	1	0
414517		0	0	0	0
393894	App shows many large bags	0	0	0	1
393044		0	0	0	0
405126		0	0	0	0
393024		0	0	0	0
411239		0	0	0	0
410559		0	0	0	0
351275		0	0	0	0
410223		0	0	0	0
407918		0	0	0	0
410591		0	0	0	0
387657		0	0	0	0
412319	Hard to read comment	0	0	0	0
405275		0	0	0	0
361987		0	0	0	0
410452		0	0	0	0"""
raw_data = data.split("\n")
raw_data = [i.split("\t") for i in raw_data]


data = {}
data['parameters'] = {
    'model': 'naveen+jingwu', 
    'prompts': 'original'
}
list_of_dicts = df.to_dict(orient='records')
list_of_dicts = [{'id': int(i[0]),
    'comment': i[1],
    'inadequate_food': int(i[2]),
    'donor_problem': int(i[3]),
    'recipient_problem': int(i[4]), 
    'info_update': int(i[5]),
} for i in raw_data[1:]]
data['records'] = []

data['annotations'] = list_of_dicts
json.dump(data,open("../../results/annotations/consensus_annotations.json","w"))